In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
cwd = os.getcwd()
file_name = "MINDsmall_train/news.tsv"
file_path =  os.path.join(cwd, file_name)

In [3]:
news_df = pd.read_csv(file_path, sep='\t',header=None, names=['NewsID', 'Category', 'SubCategory', 'Title' , 'Abstract' , 'URL', 'TitleEntities', 'AbstractEntities'])

In [4]:
news_df.head()

,NewsID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [5]:
news_df.drop(columns=['URL', 'TitleEntities' ,'AbstractEntities' ], inplace=True)

In [6]:
news_df.head()

,NewsID,Category,SubCategory,Title,Abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."


In [7]:
news_df.isna().sum()

NewsID            0
Category          0
SubCategory       0
Title             0
Abstract       2666
dtype: int64

In [8]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [108]:
from pinecone import Pinecone, ServerlessSpec

INDEX_NAME = 'pc-index-news'
pinecone = Pinecone(api_key=PINECONE_API_KEY)

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=384, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)

In [32]:
# Confirm we indeed created our  index

pinecone.list_indexes().names()

['pc-index-news']

In [33]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [12]:
from sentence_transformers import SentenceTransformer


def get_embeddings(articles, model=SentenceTransformer('all-MiniLM-L6-v2')):
   return model.encode(articles)

In [14]:
import math

# Function to check if a value is NaN
def is_nan(value):
    try:
        return math.isnan(value)
    except:
        return False

In [110]:
from tqdm.auto import tqdm, trange

CHUNK_SIZE=200
TOTAL_ROWS=10000
progress_bar = tqdm(total= TOTAL_ROWS)
use_columns = [0,1,2,3,4]
namespace = 'NewsTitles'

chunks = pd.read_csv(file_path, sep='\t',header=None,usecols=use_columns , 
                     names=['NewsID', 'Category', 'SubCategory', 'Title' , 'Abstract' ]
                     , chunksize=CHUNK_SIZE
                     ,nrows=TOTAL_ROWS 
                     )



chunk_num = 0
for chunk in chunks:
    titles = chunk['Title'].tolist()
    newsIds = chunk['NewsID'].tolist()
    cleaned_titles = [title if not is_nan(title) else "" for title in titles]
    embeddings = get_embeddings(cleaned_titles)
    prepped = [{'id':str(newsIds[i])
                , 'values':embeddings[i],
                'metadata':{'title':cleaned_titles[i] , 'NewsID':str(newsIds[i])}} 
               for i in range(0,len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
        try:
            index.upsert(prepped , namespace=namespace)
        except Exception as e:
            print(f"Error upserting chunk {chunk_num}: {e}")
        prepped = []
    progress_bar.update(len(chunk))

#final
if prepped:
    try:
        index.upsert(prepped,namespace=namespace)
    except Exception as e:
        print(f"Error upserting final chunk: {e}")

100%|██████████| 10000/10000 [24:10<00:00,  6.89it/s]


100%|██████████| 10000/10000 [07:49<00:00, 22.79it/s]

In [111]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'NewsTitles': {'vector_count': 10000}},
 'total_vector_count': 10000}

In [40]:
def get_recommendations(pinecone_index, search_term, namespace = 'NewsTitles' ,top_k=10 ):
  embed = get_embeddings(search_term)
  res = pinecone_index.query(namespace = namespace ,vector=embed.tolist(), top_k=top_k, include_metadata=True )
  return res

In [43]:
reco = get_recommendations(index, 'disease', 'NewsTitles')
for r in reco.matches:
    print(f'{r.score} : {r.id}-{r.metadata["title"]}')

0.547162294 : N36551-42 Strange Symptoms That Can Signal a Serious Disease
0.482564598 : N2360-'Ebola Is Now a Disease We Can Treat.' How a Cure Emerged From a War Zone.
0.480064034 : N688-A rare disease hardened her skin. A 'game-changing' treatment from Baptist is freeing her
0.455787212 : N55975-What to Know About the Rare Disease That Killed Bernie Sanders' Daughter-in-Law
0.439851612 : N13279-Worst Things For Your Health According to Doctors
0.439211428 : N25344-12 Diseases Doctors Can Detect Through Smell
0.433002442 : N5547-Rare Disease That Spreads from Animals to Humans Reported in Utah and Oklahoma Dogs
0.404349446 : N63102-Hospital Creates 'Cure' For Picky-Eating Children
0.392818481 : N35703-Her cancer surgery was a success. Then a genetic condition let a chemo drug ravage her body
0.389198393 : N10487-9 grim facts about the vaping illness epidemic


**Part2 : User Data**

In [46]:
file_name = "MINDsmall_train/behaviors.tsv"
file_path =  os.path.join(cwd, file_name)
behavior_df = pd.read_csv(file_path, sep='\t',header=None, names=['ImpressionID', 'UserID', 'Time', 'History' , 'Impressions' ])
behavior_df.head()

,ImpressionID,UserID,Time,History,Impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [47]:
behavior_df[behavior_df['History'].isna()]

,ImpressionID,UserID,Time,History,Impressions
28,29,U33207,11/11/2019 11:09:14 AM,NaN,N62212-0 N27521-0 N28983-0 N12028-0 N25437-0 N...
96,97,U89839,11/11/2019 4:57:51 AM,NaN,N12042-0 N35729-0 N49685-0 N62729-1
155,156,U5598,11/13/2019 2:01:40 PM,NaN,N7121-0 N37437-0 N53861-0 N45891-0 N21141-0 N5...
194,195,U19208,11/11/2019 3:57:34 PM,NaN,N23414-1 N52474-0 N6099-0 N5980-0 N1914-0
257,258,U6879,11/14/2019 5:33:52 PM,NaN,N9284-0 N23391-0 N42849-0 N33828-0 N57081-0 N2...
...,...,...,...,...,...
156850,156851,U85936,11/10/2019 8:52:02 AM,NaN,N41140-1 N4148-0
156890,156891,U4828,11/11/2019 9:41:21 AM,NaN,N8957-0 N60162-0 N16804-0 N24180-0 N31370-0 N2...
156897,156898,U38290,11/14/2019 3:50:07 PM,NaN,N59267-0 N12446-0 N53031-0 N3380-0 N56211-0 N5...
156921,156922,U64209,11/11/2019 7:11:07 AM,NaN,N35729-0 N15830-0 N48759-0 N21519-1 N27581-0 N...


In [48]:
len(behavior_df)

156965

In [49]:
behavior_df[behavior_df['UserID'].map(behavior_df['UserID'].value_counts()) >1].sort_values(by = ['UserID','ImpressionID'])

,ImpressionID,UserID,Time,History,Impressions
46639,46640,U1000,11/14/2019 1:00:56 AM,N29641 N1789 N41244,N16148-0 N43628-0 N13094-0 N32774-0 N57034-0 N...
74344,74345,U1000,11/14/2019 10:37:21 PM,N29641 N1789 N41244,N14478-0 N58264-0 N7494-0 N46917-0 N22257-0 N1...
86766,86767,U1000,11/13/2019 11:16:18 PM,N29641 N1789 N41244,N35273-0 N7618-0 N55281-0 N4021-0 N16148-0 N38...
60202,60203,U10001,11/14/2019 5:46:54 AM,N47937 N51706 N56447 N61319 N27644 N18030 N272...,N42860-0 N62750-0 N33025-0 N3167-0 N51975-0 N4...
155922,155923,U10001,11/11/2019 5:30:21 AM,N47937 N51706 N56447 N61319 N27644 N18030 N272...,N35729-0 N56598-0 N48759-0 N49685-0 N33632-0 N...
...,...,...,...,...,...
76920,76921,U9996,11/13/2019 10:07:24 AM,N28296 N4719 N60340 N31165 N8448,N287-1 N64094-0 N51048-0 N39010-0
120621,120622,U9996,11/14/2019 8:41:12 AM,N28296 N4719 N60340 N31165 N8448,N38779-0 N1539-0 N45523-0 N40109-0 N50872-0 N2...
154397,154398,U9996,11/14/2019 8:40:37 AM,N28296 N4719 N60340 N31165 N8448,N40109-0 N6477-0 N23877-0 N23446-1 N50872-0 N3...
62952,62953,U9997,11/13/2019 2:41:42 PM,N46759 N46990 N9072 N11727 N50744 N38367 N3333...,N35273-0 N4510-0 N48410-1 N29170-0 N26262-0 N5...


In [50]:
behavior_df = behavior_df.dropna(subset=['History'])

In [51]:
behavior_df['History'].isna().sum()

0

In [52]:
len(behavior_df)

153727

In [53]:
#split NewsID in history into separate rows :
split_history_df = behavior_df['History'].str.split(' ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='NewsID')

In [140]:
split_history_df.head()

,index,NewsID
0,0,N55189
1,0,N42782
2,0,N34694
3,0,N45794
4,0,N18445


In [74]:
expanded_df = behavior_df.merge(split_history_df, left_index=True, right_on='index').drop(columns=['index'])
expanded_df.head(10)


,ImpressionID,UserID,Time,History,Impressions,NewsID
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N55189
1,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N42782
2,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N34694
3,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N45794
4,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N18445
5,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N63302
6,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N10414
7,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N19347
8,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N31801
9,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,N31739


In [75]:
len(expanded_df)

5107639

In [76]:
expanded_df['History'].isna().sum()

0

In [77]:
df_merged = pd.merge(expanded_df, news_df, on='NewsID', how='left')
df_merged.head(10)

,ImpressionID,UserID,Time,History,Impressions,NewsID,Category,SubCategory,Title,Abstract
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi..."
1,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N42782,sports,baseball_mlb,Three takeaways from Yankees' ALCS Game 5 vict...,The Yankees kept hope alive thanks to some imp...
2,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N34694,tv,tvnews,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...
3,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N45794,news,newscrime,Four flight attendants were arrested in Miami'...,Four American Airlines flight attendants were ...
4,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N18445,sports,football_ncaa,Michigan sends breakup tweet to Notre Dame as ...,"Parting is such sweet sorrow, say the Wolverines."
5,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N63302,lifestyle,lifestylebuzz,This Wedding Photo of a Canine Best Man Captur...,"When Mark Doublet made his dog, Marley, the be..."
6,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N10414,movies,movienews,"Robert Evans, 'Chinatown' Producer and Paramou...","Robert Evans, the Paramount executive who prod..."
7,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N19347,news,newspolitics,Former US Senator Kay Hagan dead at 66,"Former U.S. Sen. Kay Hagan, a one-time Capitol..."
8,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N31801,news,newspolitics,Joe Biden reportedly denied Communion at a Sou...,Joe Biden has a complicated history with the C...
9,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,N31739,news,newscrime,"'Habitual stowaway' arrested again at O'Hare, ...",A woman arrested at O'Hare International Airpo...


In [78]:
len(df_merged)

5107639

In [79]:
pd.set_option('display.max_colwidth', None)
df_concat = df_merged.groupby('ImpressionID')['Title'].apply(lambda x: ';'.join(x)).reset_index()
df_concat = df_concat.rename(columns={'Title': 'Titles'})
df_concat.head(10)

ImpressionID  \
0             1   
1             2   
2             3   
3             4   
4             5   
5             6   
6             7   
7             8   
8             9   
9            10   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [80]:
len(df_concat)

153727

In [81]:
final_user_df = pd.merge(behavior_df , df_concat, on="ImpressionID", how='inner')
final_user_df.head()

ImpressionID  UserID                   Time  \
0             1  U13740  11/11/2019 9:05:58 AM   
1             2  U91836  11/12/2019 6:11:30 PM   
2             3  U73700  11/14/2019 7:01:48 AM   
3             4  U34670  11/11/2019 5:28:05 AM   
4             5   U8125  11/12/2019 4:11:21 PM   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         History  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801   
1  N31739 N6072 N63045 N23979 N35656 N43353 N8129 N1569 N17686 N13008 N21623 N6233 N14340 N48031 N62285 N44383 N23061 N16290 N6244 N45099 N58715 N59049 N7023 N50528 N42704 N46082 N8275 N15710 N59026 N8429 N30867 N56514 N19709 N31402 N31741 N54889 N9798 N62612 N2663 N16617 N6087 N13231 N63317 N61388 N59359 N51163 N30698 N34567 N54225 N32852 N55833 N64467 N3142 N13912 N29802 N44462 N29948 N4486 N5398 N14761 N47020 N65112 N31699 N37159 N61101 N14761 N3433 N10438 N61355 N21164 N22976 N2511 N48390 N58224 N48742 N35458 N24611 N37509 N21773 N41011 N19041 N25785   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                  N10732 N25792 N7563 N21087 N41087 N5445 N60384 N46616 N52500 N33164 N47289 N24233 N62058 N26378 N49475 N18870   
3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             N45729 N2203 N871 N53880 N41375 N43142 N33013 N29757 N31825 N51891   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    N10078 N56514 N14904 N33740   

                                                                                                                                                                                                                                                                                                                             

In [82]:
final_user_df['Titles'].isna().sum()

0

In [83]:
final_user_df.dtypes

ImpressionID     int64
UserID          object
Time            object
History         object
Impressions     object
Titles          object
dtype: object

In [85]:
final_user_df['Titles'] = final_user_df['Titles'].astype('string')

In [88]:
user_history_titles_df = final_user_df[['UserID' , 'Titles']]
len(user_history_titles_df)

153727

In [89]:
user_history_titles_df = user_history_titles_df.drop_duplicates()
len(user_history_titles_df)

49108

In [90]:
behavior_df['UserID'].nunique()

49108

In [91]:
CHUNK_SIZE = 200
N_ROWS = 1000
prepped = []
progress_bar = tqdm(total= N_ROWS)
namespace = 'UserHistory'

def chunk_df(df ,CHUNK_SIZE ):
    for i in range(0,N_ROWS, CHUNK_SIZE): #for i in range(0,len(df), CHUNK_SIZE)
        yield df.iloc[i:i+CHUNK_SIZE]

chunk_num = 0
for chunk in chunk_df(user_history_titles_df , CHUNK_SIZE):
    titles = chunk['Titles'].tolist()
    embeddings = get_embeddings(titles)
    prepped = [{'id':str(chunk_num*CHUNK_SIZE+i), 'values':embeddings[i],
                'metadata':{'UserID':chunk['UserID'].iloc[i]}} for i in range(0,len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
        try:
            index.upsert(prepped , namespace= namespace )
        except:
            print(f"Error upserting chunk {chunk_num}: {e}")
        prepped = []
    progress_bar.update(len(chunk))

#final
if prepped:
    try:
        index.upsert(prepped , namespace= namespace)
    except Exception as e:
        print(f"Error upserting final chunk: {e}")


100%|██████████| 1000/1000 [01:06<00:00, 15.03it/s]

In [92]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'NewsTitles': {'vector_count': 10000},
                'UserHistory': {'vector_count': 1000}},
 'total_vector_count': 11000}

In [97]:
# Given user and list of NewsIDs, rank the IDs.
final_user_df['Impression_List'] = final_user_df['Impressions'].str.replace(r'-(1|0)','',regex=True)
final_user_df['Impression_List'] = final_user_df['Impression_List'].apply(lambda x: x.split())
final_user_df.head()

ImpressionID  UserID                   Time  \
0             1  U13740  11/11/2019 9:05:58 AM   
1             2  U91836  11/12/2019 6:11:30 PM   
2             3  U73700  11/14/2019 7:01:48 AM   
3             4  U34670  11/11/2019 5:28:05 AM   
4             5   U8125  11/12/2019 4:11:21 PM   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         History  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801   
1  N31739 N6072 N63045 N23979 N35656 N43353 N8129 N1569 N17686 N13008 N21623 N6233 N14340 N48031 N62285 N44383 N23061 N16290 N6244 N45099 N58715 N59049 N7023 N50528 N42704 N46082 N8275 N15710 N59026 N8429 N30867 N56514 N19709 N31402 N31741 N54889 N9798 N62612 N2663 N16617 N6087 N13231 N63317 N61388 N59359 N51163 N30698 N34567 N54225 N32852 N55833 N64467 N3142 N13912 N29802 N44462 N29948 N4486 N5398 N14761 N47020 N65112 N31699 N37159 N61101 N14761 N3433 N10438 N61355 N21164 N22976 N2511 N48390 N58224 N48742 N35458 N24611 N37509 N21773 N41011 N19041 N25785   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                  N10732 N25792 N7563 N21087 N41087 N5445 N60384 N46616 N52500 N33164 N47289 N24233 N62058 N26378 N49475 N18870   
3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             N45729 N2203 N871 N53880 N41375 N43142 N33013 N29757 N31825 N51891   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    N10078 N56514 N14904 N33740   

                                                                                                                                                                                                                                                                                                                             

In [100]:
sample_Titles = """ 'Wheel Of Fortune' Guest Delivers Hilarious, Off The Rails Introduction;Three takeaways from Yankees' ALCS Game 5 victory over the Astros;Rosie O'Donnell: Barbara Walters Isn't 'Up to Speaking to People' Right Now;Four flight attendants were arrested in Miami's airport after bringing in thousands in cash, police say;Michigan sends breakup tweet to Notre Dame as series goes on hold;This Wedding Photo of a Canine Best Man Captures Just How Deep a Dog's Love Truly Is;Robert Evans, 'Chinatown' Producer and Paramount Chief, Dies at 89;Former US Senator Kay Hagan dead at 66;Joe Biden reportedly denied Communion at a South Carolina church because of his stance on abortion
"""

In [101]:
sample_Titles

" 'Wheel Of Fortune' Guest Delivers Hilarious, Off The Rails Introduction;Three takeaways from Yankees' ALCS Game 5 victory over the Astros;Rosie O'Donnell: Barbara Walters Isn't 'Up to Speaking to People' Right Now;Four flight attendants were arrested in Miami's airport after bringing in thousands in cash, police say;Michigan sends breakup tweet to Notre Dame as series goes on hold;This Wedding Photo of a Canine Best Man Captures Just How Deep a Dog's Love Truly Is;Robert Evans, 'Chinatown' Producer and Paramount Chief, Dies at 89;Former US Senator Kay Hagan dead at 66;Joe Biden reportedly denied Communion at a South Carolina church because of his stance on abortion\n"

In [103]:
reco = get_recommendations(index, sample_Titles, 'NewsTitles')
for r in reco.matches:
    print(f'{r.score} : {r.id}-{r.metadata["title"]}')

0.459479809 : N63302-This Wedding Photo of a Canine Best Man Captures Just How Deep a Dog's Love Truly Is
0.424786627 : N55189-'Wheel Of Fortune' Guest Delivers Hilarious, Off The Rails Introduction
0.413145512 : N27341-These Brides Got Married in the Hawaiian Jungle, and There Was a Tiny Horse Involved - It Was Wild!
0.409392208 : N651-Nine Celebrity Chef Restaurant Super Flops
0.402621478 : N57739-Conan the hero dog holds a press conference on 'SNL'
0.400774539 : N50899-Elizabeth Warren's golden retriever crashes CNN interview
0.392070025 : N35834-Groom Misses Wedding to Undergo Emergency Brain Surgery
0.382983 : N9209-'We were blindsided': NC vet and his wife deal with cancer -- and the VA
0.379593253 : N63496-Biggest Late-Night Guests Now Bring a News Angle, Not a Movie Clip
0.377434909 : N52059-The Jets are upset ESPN aired Sam Darnold's mic'd up quote about 'seeing ghosts'


In [ ]:
def rank_impressions(pinecone_index, search_term, namespace = 'NewsTitles' ,filter = {
  "genre": { "$in": ["comedy", "documentary", "drama"] }
} ):
  embed = get_embeddings(search_term)
  res = pinecone_index.query(namespace = namespace ,vector=embed.tolist(), top_k=top_k, include_metadata=True )
  return res